In [1]:
import torch
from torch_geometric.data import Data
import pandas as pd
import networkx as nx
from torch_geometric.utils.convert import from_networkx
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import graphsage_calculate_embeddings
import torch.nn.functional as F
import read_data
import test_embeddings
import locale



# Understanding the Dataset

nodes.csv: Contains the node IDs, where each ID represents a unique user on YouTube.

groups.csv:  Contains group IDs, each representing a different group or community within the YouTube platform.

edges.csv:  Comprises pairs of user IDs, with each pair indicating a friendship link between two users. For example, an entry "1,2" means that the user with ID "1" is friends with the user with ID "2".

group-edges.csv: Each line has two numbers; the first is a user ID, and the second is a group ID. This file links users to the groups they are part of.

In [2]:
# Paths to data files
nodes_path = 'datasets/YouTube-dataset/data/nodes.csv'
edges_path = 'datasets/YouTube-dataset/data/edges.csv'
groups_path = 'datasets/YouTube-dataset/data/groups.csv'
group_edges_path = 'datasets/Youtube-dataset/data/group-edges.csv'

# Create graph object and transform it to torch_geometric.data

In [ ]:
graph, labels = read_data.read_dataset_arizona_university(nodes_path, edges_path, groups_path, group_edges_path)
data = from_networkx(graph)
# TODO: error when reading data because some nodes don't have labels 

In [ ]:
# Adjusting labels and features 
data.y = torch.from_numpy(labels.astype(float))
# Add x variable which diagonal matrix with 1's as entries and size = num_nodes x num_nodes
x_diagonal = torch.eye(data.num_nodes)
data.x = x_diagonal.float()

# Define hyperparameters

In [ ]:
learning_rate = 0.0001 
aggregator = 'MeanAggregation'
epochs = 10
dropout_rate = 0.4
normalization = True 
activation_function = F.relu
bias = True
batch_size =  512
neighborhood_1 = 25
neighborhood_2 = 10
embedding_dimension = 128
hidden_layer = 512
project = False

# Obtain embedding matrix

In [ ]:
number_features, number_nodes = data.num_features, data.x.shape[0]

In [ ]:
embedding_matrix = graphsage_calculate_embeddings.compute_embedding_matrix(
    data=data,
    number_features=number_features,
    number_nodes=number_nodes,
    batch_size=batch_size,
    hidden_layer=hidden_layer,
    epochs=epochs,
    neighborhood_1=neighborhood_1,
    neighborhood_2=neighborhood_2,
    embedding_dimension=embedding_dimension,
    learning_rate=learning_rate,
    dropout_rate=dropout_rate,
    activation_function=activation_function,
    aggregator=aggregator,
    activation_before_normalization=True, 
    bias=True,
    normalize=normalization, 
    project=project
)


# Save embedding

In [ ]:
file_name = 'embeddings/youtube.pt'
torch.save(embedding_matrix, file_name)

# Testing results

# Node clasification

In [ ]:
acc, f1_macro, f1_micro = test_embeddings.test_node_classification_multi_class(embedding_matrix, data.y)
#print(f"Accuracy: {acc*100:.4f}, F1_macro: {f1_macro*100:.4f}, F1_micro: {f1_micro*100:.4f}")

In [ ]:
locale.setlocale(locale.LC_ALL, 'de_DE')

# Format the numbers with four digits after the decimal and replace the dot with a comma
formatted_acc = locale.format_string("%.4f", acc * 100).replace('.', ',')
formatted_f1_macro = locale.format_string("%.4f", f1_macro * 100).replace('.', ',')
formatted_f1_micro = locale.format_string("%.4f", f1_micro * 100).replace('.', ',')

print(f"Accuracy: {formatted_acc}, F1_macro: {formatted_f1_macro}, F1_micro: {formatted_f1_micro}")

# Link Prediction

In [ ]:
train_data, test_data = test_embeddings.train_test_split_graph(data = data, is_undirected = True) # TODO: change the is_undirected depending on graph

# Prepare edges
test_edges = test_data.edge_label_index.numpy().T
y_true = test_data.edge_label.numpy()

# Prepare embeddings
embedding_detached = embedding_matrix.detach()
embedding_np = embedding_detached.numpy()

In [ ]:
roc_auc_score = test_embeddings.k_fold_cross_validation_link_prediction(embedding_np, test_edges, y_true, k=5)


In [ ]:
formatted_score = "{:.4f}".format(roc_auc_score * 100).replace('.', ',')
print("ROC AUC Score:", formatted_score)